In [2]:
import numpy as np
import pandas as pd
import os
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

from sklearn.preprocessing import MinMaxScaler

#from dfply import arrange

from sklearn.utils import resample

In [10]:
path1 = "D:/tunc_oz/apply_model"
os.chdir(path1)

# Second path
path2 = "csv_data_r_import/cologne/scale"
os.chdir(path2)

In [3]:
def svm_fit(x, y, index_train):
    # Expand coarse grid
    coarse_grid = {'C': 2.0 ** np.arange(-4, 13, 2),
                   'gamma': 2.0 ** np.arange(-5, 4, 2)}
    
    # Coarse grid search
    svm_coarse = SVC(kernel='rbf')
    svm_coarse_cv = GridSearchCV(svm_coarse, param_grid=coarse_grid, scoring='accuracy', cv=index_train)
    svm_coarse_cv.fit(x, y)
    
    # Get best coarse grid parameters
    best_c = svm_coarse_cv.best_params_['C']
    best_gamma = svm_coarse_cv.best_params_['gamma']
    
    # Define narrow grid borders
    a_gamma = np.log2(best_gamma) - 2
    b_gamma = np.log2(best_gamma) + 2
    a_c = np.log2(best_c) - 2
    b_c = np.log2(best_c) + 2
    
    # Expand narrow grid
    narrow_grid = {'C': 2.0 ** np.arange(a_c, b_c, 0.5),
                   'gamma': 2.0 ** np.arange(a_gamma, b_gamma, 0.5)}
    
    # Narrow grid search
    svm_narrow = SVC(kernel='rbf')
    svm_narrow_cv = GridSearchCV(svm_narrow, param_grid=narrow_grid, scoring='accuracy', cv=index_train)
    svm_narrow_cv.fit(x, y)
    
    return svm_narrow_cv

# Usage example:
# svm_model = svm_fit(X_train, y_train, StratifiedKFold(n_splits=10, shuffle=True))

In [4]:
# Euclidean Distance between two points lying in the input space
def euc_dis(a, b):
    temp = 0
    for ii in range(len(a)):
        temp += (a[ii] - b[ii])**2
    return np.sqrt(temp)

# Evaluate the distance between Virtual Support Vectors and Support Vectors lying in the input space
def rem_extrem(org, VSV1, a):
    distance = pd.DataFrame(index=range(len(org)), columns=['label', 'distance'])
    distanceSVC1 = []
    distanceSVC2 = []
    
    for l in range(len(org)):
        distance.loc[l, 'label'] = str(org.iloc[l, -1])
        distance.loc[l, 'distance'] = euc_dis(org.iloc[l, :-1], VSV1.iloc[l, :-1])
    
    SVClass1 = org[org['REF'] == org['REF'].unique()[0]]
    SVClass2 = org[org['REF'] == org['REF'].unique()[1]]
    
    if len(SVClass1) > 0:
        for n in range(len(SVClass1) - 1):
            for nn in range(n, len(SVClass1) - 1):
                distanceSVC1.append(euc_dis(SVClass1.iloc[n, :-1], SVClass1.iloc[n + nn, :-1]))
        disClass1median = np.mean(distanceSVC1)
        boundClass1 = disClass1median * a
    
    if len(SVClass2) > 0:
        for n in range(len(SVClass2) - 1):
            for nn in range(n, len(SVClass2) - 1):
                distanceSVC2.append(euc_dis(SVClass2.iloc[n, :-1], SVClass2.iloc[n + nn, :-1]))
        disClass2median = np.mean(distanceSVC2)
        boundClass2 = disClass2median * a
    
    for k in range(len(org)):
        if np.isnan(distance.loc[k, 'distance']):
            VSV1.iloc[k, :] = np.nan
        else:
            if boundClass1 is not None:
                if distance.loc[k, 'label'] == org['REF'].unique()[0]:
                    if distance.loc[k, 'distance'] > boundClass1:
                        VSV1.iloc[k, :] = np.nan
            else:
                if boundClass2 is not None:
                    if distance.loc[k, 'label'] == org['REF'].unique()[1]:
                        if distance.loc[k, 'distance'] > boundClass2:
                            VSV1.iloc[k, :] = np.nan
    return VSV1

In [5]:
# Kernel distance between two points lying in the hyperspace
def kern_dis(a, b, kernelfunc):
    a = np.array(a).flatten()
    b = np.array(b).flatten()
    dk = np.sqrt(kernelfunc(a, a) + kernelfunc(b, b) - 2 * kernelfunc(a, b))
    return dk

# Evaluate the distance between Virtual Support Vectors and Support Vectors lying in the hyperspace
def rem_extrem_kerneldist(org, VSV1, a, kernelfunc):
    distance = pd.DataFrame(index=range(len(org)), columns=['label', 'distance'])
    distanceSVC1 = []
    distanceSVC2 = []
    
    for l in range(len(org)):
        distance.loc[l, 'label'] = str(org.iloc[l, -1])
        distance.loc[l, 'distance'] = kern_dis(org.iloc[l, :-1], VSV1.iloc[l, :-1], kernelfunc)
    
    SVClass1 = org[org['REF'] == org['REF'].unique()[0]]
    SVClass2 = org[org['REF'] == org['REF'].unique()[1]]
    
    if len(SVClass1) > 0:
        for n in range(len(SVClass1) - 1):
            for nn in range(n, len(SVClass1) - 1):
                distanceSVC1.append(kern_dis(SVClass1.iloc[n, :-1], SVClass1.iloc[n + nn, :-1], kernelfunc))
        disClass1median = np.mean(distanceSVC1)
        boundClass1 = disClass1median * a
    
    if len(SVClass2) > 0:
        for n in range(len(SVClass2) - 1):
            for nn in range(n, len(SVClass2) - 1):
                distanceSVC2.append(kern_dis(SVClass2.iloc[n, :-1], SVClass2.iloc[n + nn, :-1], kernelfunc))
        disClass2median = np.mean(distanceSVC2)
        boundClass2 = disClass2median * a
    
    for k in range(len(org)):
        if np.isnan(distance.loc[k, 'distance']):
            VSV1.iloc[k, :] = np.nan
        else:
            if boundClass1 is not None:
                if distance.loc[k, 'label'] == org['REF'].unique()[0]:
                    if distance.loc[k, 'distance'] > boundClass1:
                        VSV1.iloc[k, :] = np.nan
            else:
                if boundClass2 is not None:
                    if distance.loc[k, 'label'] == org['REF'].unique()[1]:
                        if distance.loc[k, 'distance'] > boundClass2:
                            VSV1.iloc[k, :] = np.nan
    return VSV1

In [6]:
def pred_one(model, data_point):
    # Extract necessary components from the SVM model
    support_vectors = model.n_support_
    kernel_function = model.kernel
    coefficients = model.dual_coef_.ravel()
    intercept = model.intercept_
    
    # Initialize prediction variable
    prediction = 0
    
    # Iterate over each support vector
    for j in range(len(support_vectors)):
        # Compute kernel function value between the j-th support vector and the data point
        kernel_value = kernel_function(data_point.reshape(1, -1), model.support_vectors_[j, :].reshape(1, -1))
        
        # Multiply kernel value by the corresponding coefficient and add to prediction
        weighted_value = kernel_value * coefficients[j]
        prediction += weighted_value
    
    # Subtract intercept to get the final prediction
    final_prediction = prediction - intercept
    
    return final_prediction

def uncertainty_dist_v2_2(org, samp):
    distance = pd.DataFrame(columns=['control_label', 'distance'], index=range(len(samp)))
    
    for k in range(len(samp)):
        distance.loc[k, 'distance'] = np.sign(pred_one(org.finalModel, samp.iloc[k, :-1])) * \
                                      np.where(pred_one(org.finalModel, samp.iloc[k, :-1]) > 0, 1, -1)
    
    # Normalize distance
    preProc = preprocessing.MinMaxScaler()
    preProc.fit(distance[['distance']])
    normdistance = preProc.transform(distance[['distance']])
    
    samp['normdistance'] = normdistance
    
    return samp

In [7]:
def alter_labels(distance_data, ref):
    # Merge features and original labels
    ref_added = pd.concat([distance_data, ref], axis=1)
    # Order by most uncertain samples
    ref_added_or = ref_added.sort_values(by='distance')
    # Re-label most uncertain n number of samples
    ref_added_or.iloc[:250, -1] = ref_added_or.iloc[:250, -2]
    ref_added_or.iloc[:250, -2] = 1.0
    # Re-order dataset by its index
    ref_added_or['index'] = range(len(ref_added_or))
    ref_added_reor = ref_added_or.sort_values(by='index')
    
    # Extract labels for prediction
    labels = ref_added_reor.iloc[:, -5]
    return labels

def ExCsvMSD(datadase, filename=None):
    # Convert to numpy array
    datadase = np.array(datadase)
    n = datadase.shape[1]
    MSDdata = np.empty((2, n), dtype=float)
    
    MSDdata[0, :] = np.mean(datadase, axis=0)
    MSDdata[1, :] = np.std(datadase, axis=0)
    
    MSDdata_final = np.vstack((datadase, MSDdata))
    
    # Export final mean and standard deviation to .csv-file
    if filename is not None:
        pd.DataFrame(MSDdata_final).to_csv(filename, index=False, header=False)
    
    return MSDdata_final

In [11]:
inputPath = "cologne_res_100_L2-L13.csv"
sMax = 1000
bound = [0.3, 0.6, 0.9]
boundMargin = [1.5, 1.0, 0.5]
sampleSizesPor = [40, 25, 16, 12, 10, 8, 6, 4, 3, 2, 1]
colheader = ["40", "25", "16", "12", "10", "8", "6", "4", "3", "2", "1"]
sindexSVMDATA = 37
numFeat = 18
eindexSVMDATA = sindexSVMDATA + numFeat - 1
objInfoNames = ["Lx_g_comp", "Lx_g_elfi", "Lx_g_refi", "Lx_g_roun", "Lx_g_shin",
                "Lx_m_bl", "Lx_m_gr", "Lx_m_ndvi", "Lx_m_nir", "Lx_m_re",
                "Lx_sd_bl", "Lx_sd_gr", "Lx_sd_ndvi", "Lx_sd_nir", "Lx_sd_re",
                "Lx_t_diss", "Lx_t_hom", "Lx_t_mean",
                "label"]
columnClass = [None] * 217 + ["factor", "integer"]

# Import data
generalDataPool = pd.read_csv(inputPath, header=0, sep=";", dtype=str, na_values=None)
generalDataPool.dropna(subset=["REF"], inplace=True)  # Remove rows with missing REF values
generalDataPool["REF"] = pd.Categorical(generalDataPool["REF"])

# Transform to 2-Class-Case "Bushes Trees" VS rest
first_label_class = generalDataPool["REF"].cat.categories[0]  # Note that the first record is of class "bushes trees"
generalDataPool["REF"] = generalDataPool["REF"].apply(lambda x: first_label_class if x == first_label_class else "other")
generalDataPool["REF"] = pd.Categorical(generalDataPool["REF"])

data = generalDataPool.iloc[:, sindexSVMDATA:eindexSVMDATA + 1]
REF = generalDataPool.iloc[:, -1]
data_with_label = pd.concat([data, REF], axis=1)
data_label = data_with_label.iloc[:, -1]

In [ ]:
normalizedFeat

In [24]:
generalDataPool_columns = normalizedFeat.columns

In [26]:
generalDataPool.dtypes

L02_G_COMP    float64
L02_G_EFIT    float64
L02_G_RFIT    float64
L02_G_ROUN    float64
L02_G_SHIN    float64
               ...   
L13_T_DISS    float64
L13_T_HOM     float64
L13_T_MEA     float64
REF            object
USE            object
Length: 218, dtype: object

In [25]:
converters = {col: lambda x: float(x.replace(',', '.')) for col in generalDataPool_columns}
generalDataPool = pd.read_csv(inputPath, header=0, sep=";", na_values=None, converters=converters)


C:\Users\Utente\AppData\Local\Temp\ipykernel_14380\963267354.py:2: ParserWarning: Both a converter and dtype were specified for column L02_G_COMP - only the converter will be used.
  generalDataPool = pd.read_csv(inputPath, header=0, sep=";", dtype=str, na_values=None, converters=converters)
C:\Users\Utente\AppData\Local\Temp\ipykernel_14380\963267354.py:2: ParserWarning: Both a converter and dtype were specified for column L02_G_EFIT - only the converter will be used.
  generalDataPool = pd.read_csv(inputPath, header=0, sep=";", dtype=str, na_values=None, converters=converters)
C:\Users\Utente\AppData\Local\Temp\ipykernel_14380\963267354.py:2: ParserWarning: Both a converter and dtype were specified for column L02_G_RFIT - only the converter will be used.
  generalDataPool = pd.read_csv(inputPath, header=0, sep=";", dtype=str, na_values=None, converters=converters)
C:\Users\Utente\AppData\Local\Temp\ipykernel_14380\963267354.py:2: ParserWarning: Both a converter and dtype were specifi

In [18]:
normalizedFeat['L02_G_COMP'] = normalizedFeat['L02_G_COMP'].replace(',', '.', regex=True)

normalizedFeat['L02_G_COMP'] = normalizedFeat['L02_G_COMP'].astype(float)

In [ ]:
normalizedFeat['L02_G_EFIT'] = normalizedFeat['L02_G_EFIT'].apply(lambda x: pd.to_numeric(x.str.replace(',', '.'), errors='coerce'))

In [ ]:
normalizedFeat = normalizedFeat.replace(',', '.', regex=True)

normalizedFeat = normalizedFeat.astype(float)

In [32]:
normalizedFeat = generalDataPool.iloc[:, :-2]
normalizedLabelUSE = generalDataPool.iloc[:, -2:]

# Scaling
preProc = MinMaxScaler()
normalizedFeatBase = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, sindexSVMDATA:eindexSVMDATA + 1]), columns=objInfoNames[:-1])

# Apply range of basemodel to all levels
normalizedFeat2 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, :numFeat]), columns=objInfoNames[:-1])
normalizedFeat3 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, numFeat:(2 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat5 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (3 * numFeat):(4 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat6 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (4 * numFeat):(5 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat7 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (5 * numFeat):(6 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat8 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (6 * numFeat):(7 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat9 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (7 * numFeat):(8 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat10 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (8 * numFeat):(9 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat11 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (9 * numFeat):(10 * numFeat)]), columns=objInfoNames[:-1])

# Recombine normalized sets to one data frame
normalizedDataPoolAllLev = pd.concat([normalizedFeat2, normalizedFeat3, normalizedFeatBase, normalizedFeat5,
                                      normalizedFeat6, normalizedFeat7, normalizedFeat8, normalizedFeat9,
                                      normalizedFeat10, normalizedFeat11, normalizedLabelUSE], axis=1)

# Remove used temporary variables
del normalizedFeat, normalizedFeat2, normalizedFeat3, normalizedFeatBase, normalizedFeat5, normalizedFeat6
del normalizedFeat7, normalizedFeat8, normalizedFeat9, normalizedFeat10, normalizedFeat11


In [35]:
# Split data into test, train, and validate data
trainDataPoolAllLev, testDataAllLev, validateDataAllLev = [df for _, df in normalizedDataPoolAllLev.groupby('USE')]
trainDataPoolAllLev = trainDataPoolAllLev.iloc[:, :-1]
testDataAllLev = testDataAllLev.iloc[:, :-1]
validateFeatAllLev = validateDataAllLev.iloc[:, :-1]
validateLabels = validateDataAllLev.iloc[:, -1]

# Order train data pool by class label in alphabetical order
trainDataPoolAllLev = trainDataPoolAllLev.sort_values(by=trainDataPoolAllLev.columns[-1])

# Current training data-set, updated (refreshed) after each iteration
trainDataCur = trainDataPoolAllLev.copy()
testDataCur = testDataAllLev.copy()

# Set randomized seed for the random sampling procedure
seed = 5

# Initial seed value for randomized sampling
seed += np.random.randint(1, 101)

# Definition of apriori-probabilities
pA = pB = pC = pD = pE = pF = 1 / 6

# Definition of training sample set sizes S [% of max. sample size]
sCur = sMax * (sampleSizesPor[0] / 100)
# Definition of sample shares
nA, nB, nC, nD, nE, nF = [round(sCur * p) for p in [pA, pB, pC, pD, pE, pF]]
shares = [nA, nB, nC, nD, nE, nF]

# Set randomized seed for the random sampling procedure
np.random.seed(seed)

# Definition of sampling configuration (strata: random sampling without replacement)
stratSamp = trainDataCur.groupby('REF').apply(lambda x: x.sample(shares, replace=False)).reset_index(drop=True)

# Get samples of trainDataCur and set trainDataCur new
samplesID = stratSamp.iloc[:,181]
trainDataCurRemaining = trainDataCur.drop(samplesID)

# Split test feat from test label for later join with trainData
trainFeat = stratSamp.iloc[:, :-1]
trainLabels = stratSamp.iloc[:, -1]




# Subset for each outer iteration test data to speed up computing
testDataCur = testDataCur.sort_values(by=testDataCur.columns[-1])
stratSamp = testDataCur.groupby('REF').apply(lambda x: x.sample(shares, replace=False)).reset_index(drop=True)

# Split test feat from test label for later join with trainData
testFeat = stratSamp.iloc[:, :-1]
testLabels = stratSamp.iloc[:, -1]

# Subset on base level
testFeatsub = testFeat.iloc[:, sindexSVMDATA:eindexSVMDATA+1]

# TrainData index to split between train and test in svmFit
countTrainData = trainFeat.shape[0]
indexTrainData = [list(range(1, countTrainData + 1))]

# SVM base for invariants

# Subset on L_4
trainFeat = trainFeat.iloc[:, sindexSVMDATA:eindexSVMDATA+1]

# Join train and test data (separable through indexTrainData in svmFit)
tuneFeat = pd.concat([trainFeat, testFeatsub], axis=0)
tuneLabel = np.concatenate((trainLabels.values, testLabels.values))


In [ ]:
validateFeatsub = validateFeatAllLev.iloc[:, sindexSVMDATA:eindexSVMDATA+1]

# SVM parameter tuning
tunedSVM = SVC(kernel='linear')
tunedSVM.fit(tuneFeat, tuneLabel)

# Predict labels of test data
predLabelsSVM = tunedSVM.predict(validateFeatsub)

# Accuracy assessment
accSVM = accuracy_score(validateLabels, predLabelsSVM)
print(accSVM)

# VSVM on all Level SV
SVindex = tunedSVM.support_  # Indices of support vectors
SVtotal = trainDataCur.iloc[SVindex, :]  # Get support vectors

SVL2 = trainDataCur.iloc[SVindex, sindexSVMDATA - 2 * numFeat:sindexSVMDATA - numFeat ]
SVL3 = trainDataCur.iloc[SVindex, sindexSVMDATA - numFeat:sindexSVMDATA ]

SVL5 = trainDataCur.iloc[SVindex, sindexSVMDATA + numFeat:sindexSVMDATA + 2 * numFeat ]
SVL6 = trainDataCur.iloc[SVindex, sindexSVMDATA + 2 * numFeat:sindexSVMDATA + 3 * numFeat ]
SVL7 = trainDataCur.iloc[SVindex, sindexSVMDATA + 3 * numFeat:sindexSVMDATA + 4 * numFeat ]
SVL8 = trainDataCur.iloc[SVindex, sindexSVMDATA + 4 * numFeat:sindexSVMDATA + 5 * numFeat ]
SVL9 = trainDataCur.iloc[SVindex, sindexSVMDATA + 5 * numFeat:sindexSVMDATA + 6 * numFeat ]
SVL10 = trainDataCur.iloc[SVindex, sindexSVMDATA + 6 * numFeat:sindexSVMDATA + 7 * numFeat ]
SVL11 = trainDataCur.iloc[SVindex, sindexSVMDATA + 7 * numFeat:sindexSVMDATA + 8 * numFeat ]

# Bind original SV with modified to new train data set
SVinvar = pd.concat([SVtotal, SVL2, SVL3, SVL5, SVL6, SVL7, SVL8, SVL9, SVL10, SVL11])

# Split for training to feature and label
trainFeatVSVM = SVinvar.iloc[:, :-1]
trainLabelsVSVM = SVinvar.iloc[:, -1]

# Get list with index of train data to split between train and test in svmFit
countTrainData = SVinvar.shape[0]
indexTrainData = [list(range(1, countTrainData + 1))]

# Join of train and test test data (through indexTrainData in svmFit separable)
tuneFeatVSVM = pd.concat([trainFeatVSVM, testFeatsub], axis=0)
tuneLabelsVSVM = np.concatenate((trainLabelsVSVM.values, testLabels.values))

# VSVM parameter tuning
tunedVSVM = SVC(kernel='linear')
tunedVSVM.fit(tuneFeatVSVM, tuneLabelsVSVM)

# Run classification and accuracy assessment for modified SV
predLabelsVSVM = tunedVSVM.predict(validateFeatsub)
accVSVM = accuracy_score(validateLabels, predLabelsVSVM)
print(accVSVM)


In [ ]:


# VSVM - EVALUATION of all Level VSV
actKappa = 0
bestFittingModel = None

# Iteration over bound to test different bound thresholds determining the radius of acceptance
for jj, bound_val in enumerate(bound, start=1):
    SVinvarRadi_list = []
    
    # Iterating over boundMargin to test different thresholds on margin distance
    for kk, bound_margin_val in enumerate(boundMargin, start=1):
        SVinvar_list = []
        
        # Iterate over SVinvarRadi and evaluate distance to hyperplane
        for m in range(len(SVinvarRadi)):
            signa = pred_one(tunedSVM.finalModel, SVinvarRadi[m, :-1])
            if SVinvarRadi[m, -1] == levels(generalDataPool.REF)[0]:
                if -bound_margin_val < signa < bound_margin_val:
                    SVinvar_list.append(SVinvarRadi[m, :])
            else:
                if -bound_margin_val < signa < bound_margin_val:
                    SVinvar_list.append(SVinvarRadi[m, :])

        SVinvar = pd.DataFrame(SVinvar_list, columns=objInfoNames)
        
        # Merge elected VSV with original SV
        SVinvar_org = pd.concat([SVtotal, SVinvar])

        # Split for training to feature and label
        trainFeatVSVM = SVinvar_org.iloc[:, :-1]
        trainLabelsVSVM = SVinvar_org.iloc[:, -1]

        # Get list with index of trainData to split between train and test in svmFit
        countTrainData = SVinvar_org.shape[0]
        indexTrainData = [list(range(1, countTrainData + 1))]

        # Join of train and test data (through indexTrainData in svmFit separable)
        names = objInfoNames[:-1]
        tuneFeatVSVM = pd.concat([trainFeatVSVM, testFeatsub], axis=0)
        tuneFeatVSVM.columns = names
        tuneLabelsVSVM = np.concatenate((trainLabelsVSVM.values, testLabels.values))

        ######################################## VSVM control parameter tuning ########################################
        tunedVSVM = SVC(kernel='linear')
        tunedVSVM.fit(tuneFeatVSVM, tuneLabelsVSVM)

        # Get the best fitting model based on Kappa
        if actKappa < tunedVSVM.resample.Kappa:
            bestFittingModel = tunedVSVM
            actKappa = tunedVSVM.resample.Kappa

# Run classification and accuracy assessment for the best bound setting
# Predict labels of test data
predLabelsVSVMsum = bestFittingModel.predict(validateFeatsub)

# Accuracy assessment
accVSVM_SL = accuracy_score(validateLabels, predLabelsVSVMsum)


In [ ]:



# Balanced & Random unlabeled samples
# Balanced samples

# Definition of sampling configuration (strata: random sampling without replacement)
stratSampRemaining_b = resample(trainDataCurRemaining, n_samples=[b, b, b, b, b, b], replace=False)
samplesRemaining_b = trainDataCurRemaining.iloc[stratSampRemaining_b]

trainDataCurRemaining_b = samplesRemaining_b.iloc[:, :-1]
trainDataCurRemainingsub_b = trainDataCurRemaining_b.iloc[:, sindexSVMDATA:eindexSVMDATA]
REF_b = bestFittingModel.predict(trainDataCurRemainingsub_b)

SVindexUn_b = np.arange(1, len(trainDataCurRemainingsub_b) + 1)
SVtotalUn_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA:eindexSVMDATA]
SVtotalUn_b['REF'] = REF_b

SVL2Un_b = trainDataCurRemaining.iloc[SVindexUn_b - 1, sindexSVMDATA - 2*numFeat:sindexSVMDATA - numFeat - 1].copy()
SVL2Un_b['REF'] = REF_b
SVL3Un_b = trainDataCurRemaining.iloc[SVindexUn_b - 1, sindexSVMDATA - numFeat:sindexSVMDATA - 1].copy()
SVL3Un_b['REF'] = REF_b
SVL5Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + numFeat:sindexSVMDATA + 2*numFeat - 1].copy()
SVL5Un_b['REF'] = REF_b
SVL6Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + 2*numFeat:sindexSVMDATA + 3*numFeat - 1].copy()
SVL6Un_b['REF'] = REF_b
SVL7Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + 3*numFeat:sindexSVMDATA + 4*numFeat - 1].copy()
SVL7Un_b['REF'] = REF_b
SVL8Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + 4*numFeat:sindexSVMDATA + 5*numFeat - 1].copy()
SVL8Un_b['REF'] = REF_b
SVL9Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + 5*numFeat:sindexSVMDATA + 6*numFeat - 1].copy()
SVL9Un_b['REF'] = REF_b
SVL10Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + 6*numFeat:sindexSVMDATA + 7*numFeat - 1].copy()
SVL10Un_b['REF'] = REF_b
SVL11Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + 7*numFeat:sindexSVMDATA + 8*numFeat - 1].copy()
SVL11Un_b['REF'] = REF_b

SVinvarUn_b = pd.concat([SVtotalUn_b, SVL2Un_b, SVL3Un_b, SVL5Un_b, SVL6Un_b, SVL7Un_b, SVL8Un_b, SVL9Un_b, SVL10Un_b, SVL11Un_b])

# Balanced Unlabeled samples

actKappa = 0

for jj in range(len(bound)):
    SVinvarRadiUn_b_list = []
    for m in range(len(SVinvarRadiUn_b)):
        signa = pred_one(tunedSVM.finalModel, SVinvarRadiUn_b.iloc[m, :-1])
        if SVinvarRadiUn_b.iloc[m, -1] == levels(generalDataPool.REF)[0]:
            if -bound_margin_val < signa < bound_margin_val:
                SVinvarRadiUn_b_list.append(SVinvarRadiUn_b.iloc[m, :])
        else:
            if -bound_margin_val < signa < bound_margin_val:
                SVinvarRadiUn_b_list.append(SVinvarRadiUn_b.iloc[m, :])

    SVinvarUn_b = pd.DataFrame(SVinvarRadiUn_b_list, columns=objInfoNames)
    
    SVinvar_orgUn_b = pd.concat([SVtotal, SVinvarUn_b])

    trainFeatVSVMUn_b = SVinvar_orgUn_b.iloc[:, :-1]
    trainLabelsVSVMUn_b = SVinvar_orgUn_b.iloc[:, -1]

    countTrainDataUn_b = SVinvar_orgUn_b.shape[0]
    indexTrainDataUn_b = [list(range(1, countTrainDataUn_b + 1))]

    names = objInfoNames[:-1]
    tuneFeatVSVMUn_b = pd.concat([trainFeatVSVMUn_b, testFeatsub], axis=0)
    tuneFeatVSVMUn_b.columns = names
    tuneLabelsVSVMUn_b = np.concatenate((trainLabelsVSVMUn_b.values, testLabels.values))

    tunedVSVMUn_b = SVC(kernel='linear')
    tunedVSVMUn_b.fit(tuneFeatVSVMUn_b, tuneLabelsVSVMUn_b)

    if actKappa < tunedVSVMUn_b.resample.Kappa:
        bestFittingModelUn_b = tunedVSVMUn_b
        actKappa = tunedVSVMUn_b.resample.Kappa

# Run classification and accuracy assessment for the best bound setting
predLabelsVSVMsumUn_b = bestFittingModelUn_b.predict(validateFeatsub)

# Accuracy assessment
accVSVM_SL_Un_b = accuracy_score(validateLabels, predLabelsVSVMsumUn_b)


In [ ]:
# Add predicted labels to the features data set
predLabelsVSVMsumUn_unc = pd.concat([validateFeatsub, pd.DataFrame(predLabelsVSVMsumUn_b, columns=["Predicted_Labels"])], axis=1)
predLabelsVSVMsumUn_unc.columns = objInfoNames

# Calculate uncertainty of the samples by selecting SV's and data set
normdistvsvm_sl_un = uncertainty_dist_v2_2(bestFittingModelUn_b, predLabelsVSVMsumUn_unc)

# Alter labels
predlabels_vsvm_Slu = alter_labels(normdistvsvm_sl_un, validateLabels)

# Accuracy assessment
accVSVM_SL_Un_b_ad = accuracy_score(validateLabels, predlabels_vsvm_Slu)
